In [0]:
%fs rm dbfs:/FileStore/shared_uploads/ph_data_assignment/sfConfig.config



res12: Boolean = true

In [0]:
dbutils.fs.cp("FileStore/shared_uploads/ph_data_assignment/sfConfig.config", "file:///tmp/sfConfig.config")

Out[126]: True

In [0]:
## read all the credentials using config file
from configparser import ConfigParser

config_object = ConfigParser()
config_object.read("/tmp/sfConfig.config")

sfInfo = config_object["sfInfo"]
print(sfInfo["url"])
sfOptions = {
  "sfUrl" : sfInfo["url"],
  "sfUser" : sfInfo["sfUser"],
  "sfPassword" : sfInfo["password"],
  "sfDatabase" : sfInfo["database"],
  "sfSchema" : sfInfo["schema"],
  "sfWarehouse" : sfInfo["warehouse"]
}

https://ve08383.us-central1.gcp.snowflakecomputing.com


In [0]:
%fs rm dbfs:/FileStore/shared_uploads/ph_data_assignment/flights/partition_08-1.csv

res0: Boolean = true

In [0]:
df_airlines=spark.read.format("csv").options(header=True,inferSchema=True).load("dbfs:/FileStore/shared_uploads/ph_data_assignment/airlines.csv")
df_airlines.cache()
#df_airlines.show(10,False)

Out[35]: DataFrame[IATA_CODE: string, AIRLINE: string]

In [0]:
df_airports=spark.read.format("csv").options(header=True,inferSchema=True).load("dbfs:/FileStore/shared_uploads/ph_data_assignment/airports.csv")
df_airports.cache()
#df_airports.show(10,False)

Out[36]: DataFrame[IATA_CODE: string, AIRPORT: string, CITY: string, STATE: string, COUNTRY: string, LATITUDE: double, LONGITUDE: double]

In [0]:
df_flights=spark.read.format("csv").options(header=True,inferSchema=True).load("dbfs:/FileStore/shared_uploads/ph_data_assignment/flights/*.csv")
#df_flights.show(10,False)
df_flights.cache()

Out[37]: DataFrame[YEAR: int, MONTH: int, DAY: int, DAY_OF_WEEK: int, AIRLINE: string, FLIGHT_NUMBER: int, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: int, DEPARTURE_TIME: int, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: int, SCHEDULED_TIME: int, ELAPSED_TIME: int, AIR_TIME: int, DISTANCE: int, WHEELS_ON: int, TAXI_IN: int, SCHEDULED_ARRIVAL: int, ARRIVAL_TIME: int, ARRIVAL_DELAY: int, DIVERTED: int, CANCELLED: int, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: int, SECURITY_DELAY: int, AIRLINE_DELAY: int, LATE_AIRCRAFT_DELAY: int, WEATHER_DELAY: int]

In [0]:
#Total number of flights by airline and airport on a monthly basis
#Find the origin/destination airport and airlines, count the flights then
from pyspark.sql.functions import *
df_dest_flight=df_flights.select("year","month",col("DESTINATION_AIRPORT").alias("airport_code"),col("AIRLINE").alias("airline_code"))
df_origin_flight=df_flights.select("year","month",col("ORIGIN_AIRPORT").alias("airport_code"),col("AIRLINE").alias("airline_code"))
df_tot_flight=df_dest_flight.unionAll(df_origin_flight)
df_tot_flight_tmp=df_tot_flight.groupBy("year","month","airport_code","airline_code").agg(count("airport_code").alias("total"))
df_tot_flight=df_tot_flight_tmp.alias('a').join(df_airlines.alias('b')).where("a.airline_code=b.IATA_CODE").join(df_airports.alias('c')).where("a.airport_code=c.IATA_CODE").orderBy("year","month","AIRLINE","AIRPORT","total").select("year","month","AIRLINE","AIRPORT","total")
#print("total no of records - ",df_tot_flight.count())
display(df_tot_flight)


year,month,AIRLINE,AIRPORT,total
2015,1,Alaska Airlines Inc.,Adak Airport,18
2015,1,Alaska Airlines Inc.,Albuquerque International Sunport,62
2015,1,Alaska Airlines Inc.,Austin-Bergstrom International Airport,62
2015,1,Alaska Airlines Inc.,Baltimore-Washington International Airport,62
2015,1,Alaska Airlines Inc.,Bellingham International Airport,189
2015,1,Alaska Airlines Inc.,Bethel Airport,164
2015,1,Alaska Airlines Inc.,Bob Hope Airport (Hollywood Burbank Airport),186
2015,1,Alaska Airlines Inc.,Chicago O'Hare International Airport,198
2015,1,Alaska Airlines Inc.,Dallas/Fort Worth International Airport,211
2015,1,Alaska Airlines Inc.,Deadhorse Airport (Prudhoe Bay Airport),160


In [0]:
#On-time percentage of each airline for the year 2015
df_yr_data=df_flights.select("airline").where("year='2015'")
df_tot_data=df_yr_data.groupBy("airline").agg(count("airline").alias("tot_cnt"))
df_delay_data=df_yr_data.where("NVL(ARRIVAL_DELAY,0)<>0  OR NVL(DEPARTURE_DELAY,0)<>0").groupBy("airline").agg(count("airline").alias("delay_cnt"))
df_pre_final=df_tot_data.alias("a").join(df_delay_data.alias("b"),"airline").selectExpr("airline as airline_code","round(((tot_cnt-delay_cnt)*100)/(tot_cnt),3) as on_time_perc")
df_on_time=df_pre_final.join(df_airlines,df_pre_final.airline_code==df_airlines.IATA_CODE,"inner").select("airline","on_time_perc").orderBy("airline")
display(df_on_time)

airline,on_time_perc
Alaska Airlines Inc.,0.563
American Airlines Inc.,1.768
American Eagle Airlines Inc.,5.635
Atlantic Southeast Airlines,3.016
Delta Air Lines Inc.,0.761
Frontier Airlines Inc.,0.796
Hawaiian Airlines Inc.,0.598
JetBlue Airways,2.14
Skywest Airlines Inc.,1.879
Southwest Airlines Co.,1.644


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
#Airlines with the largest number of delays
#For airline with max delay. Delay canbe +/- so excluded with 0
df_pre_final=df_flights.where("NVL(ARRIVAL_DELAY,0)=0").select(col("airline").alias("airline_code")).groupBy("airline_code").agg(count("airline_code").alias("cnt"))
wind_fun=Window.orderBy(col("cnt").desc())
df_largest_delay_airline=df_pre_final.withColumn("rn",row_number().over(wind_fun)).where("rn=1").join(df_airlines,df_airlines.IATA_CODE==df_pre_final.airline_code).selectExpr("airline","cnt")
display(df_largest_delay_airline.show())

+--------------------+-----+
|             airline|  cnt|
+--------------------+-----+
|Southwest Airline...|38898|
+--------------------+-----+



In [0]:
#Cancellation reasons by airport
from pyspark.sql.functions import when,coalesce
df_canc=df_flights.where("CANCELLED=1")
df_src_canc=df_canc.selectExpr("CANCELLATION_REASON","ORIGIN_AIRPORT as airport_cd")
df_dest_canc=df_canc.selectExpr("CANCELLATION_REASON","DESTINATION_AIRPORT as airport_cd")
df_union_canc=df_src_canc.unionAll(df_dest_canc).distinct()
df_derive_canc_reason=df_union_canc.withColumn("CANCELLATION_REASON",when(df_union_canc.CANCELLATION_REASON=='A','Airline/Carrier')
.when(df_union_canc.CANCELLATION_REASON=='B','Weather')
.when(df_union_canc.CANCELLATION_REASON=='C','National Air System')
.when(df_union_canc.CANCELLATION_REASON=='D','Security')
.otherwise( df_union_canc.CANCELLATION_REASON)                                                                 ).join(df_airports,df_airports.IATA_CODE==df_union_canc.airport_cd).select("airport","CANCELLATION_REASON").orderBy("airport")
display(df_derive_canc_reason)

airport,CANCELLATION_REASON
Aberdeen Regional Airport,Weather
Aberdeen Regional Airport,Airline/Carrier
Abilene Regional Airport,Weather
Abilene Regional Airport,Airline/Carrier
Abraham Lincoln Capital Airport,National Air System
Abraham Lincoln Capital Airport,Airline/Carrier
Abraham Lincoln Capital Airport,Weather
Adak Airport,Weather
Akron-Canton Regional Airport,National Air System
Akron-Canton Regional Airport,Airline/Carrier


In [0]:
#Delay reasons by airport
from pyspark.sql.functions import when,coalesce,lit
df_delay_rsn=df_flights.withColumn("type_of_delay",when(coalesce(df_flights.AIR_SYSTEM_DELAY,lit('0'))!=0,'AIR_SYSTEM_DELAY')
                                  .when(coalesce(df_flights.SECURITY_DELAY,lit('0'))!=0,'SECURITY_DELAY')
                                  .when(coalesce(df_flights.AIRLINE_DELAY,lit('0'))!=0,'AIRLINE_DELAY')
                                  .when(coalesce(df_flights.LATE_AIRCRAFT_DELAY,lit('0'))!=0,'LATE_AIRCRAFT_DELAY')
                                  .when(coalesce(df_flights.WEATHER_DELAY,lit('0'))!=0,'WEATHER_DELAY')
                                   .otherwise('NO_DELAY')
                                  ).select("ORIGIN_AIRPORT","DESTINATION_AIRPORT","type_of_delay")

df_src_delay_rsn=df_delay_rsn.selectExpr ("ORIGIN_AIRPORT as airport_code","type_of_delay")
df_dest_delay_rsn=df_delay_rsn.selectExpr("DESTINATION_AIRPORT as airport_code","type_of_delay")
df_delay_rsn=df_src_delay_rsn.unionAll(df_dest_delay_rsn).distinct()
df_final_delay_rsn=df_delay_rsn.join(df_airports,df_delay_rsn.airport_code==df_airports.IATA_CODE).select("airport","type_of_delay")
display(df_final_delay_rsn)


airport,type_of_delay
St. Louis International Airport at Lambert Field,NO_DELAY
Southwest Florida International Airport,AIR_SYSTEM_DELAY
General Wayne A. Downing Peoria International Airport,AIR_SYSTEM_DELAY
Albert J. Ellis Airport,AIR_SYSTEM_DELAY
Jackson Hole Airport,LATE_AIRCRAFT_DELAY
Laramie Regional Airport,LATE_AIRCRAFT_DELAY
Greater Rochester International Airport,WEATHER_DELAY
Ted Stevens Anchorage International Airport,NO_DELAY
Durango-La Plata County Airport,NO_DELAY
McAllen-Miller International Airport (McAllen Miller International),NO_DELAY


In [0]:
#Airline with the most unique routes
df_rout1=df_flights.selectExpr("ORIGIN_AIRPORT as src","DESTINATION_AIRPORT as dest","airline as airline_cd")
#df_rout1
df_rout2=df_flights.selectExpr("DESTINATION_AIRPORT as src","ORIGIN_AIRPORT as dest","airline as airline_cd")
#df_rout2
df_all_routes=df_rout1.unionAll(df_rout2).groupBy("src","dest","airline_cd").agg(count("src").alias("cnt")).orderBy("cnt","airline_cd")
df_final_uniq_routes=df_all_routes.join(df_airlines,df_all_routes.airline_cd==df_airlines.IATA_CODE).select("airline","cnt").distinct().orderBy("cnt","Airline")
display(df_final_uniq_routes)


airline,cnt
American Airlines Inc.,1
American Eagle Airlines Inc.,1
Atlantic Southeast Airlines,1
Delta Air Lines Inc.,1
Frontier Airlines Inc.,1
Skywest Airlines Inc.,1
US Airways Inc.,1
United Air Lines Inc.,1
Virgin America,1
American Airlines Inc.,2


In [0]:

#Airline with the most unique routes
df_final_uniq_routes.write.format("snowflake").options(**sfOptions).option("dbtable","most_uniq_routes_aline").mode("Overwrite").save()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2878506191077943> in <module>
      1 #Airline with the most unique routes
----> 2 df_final_uniq_routes.write.format("snowflake").options(**sfOptions).option("dbtable","most_uniq_routes_aline").mode("Overwrite").save()

/databricks/spark/python/pyspark/sql/readwriter.py in save(self, path, format, mode, partitionBy, **options)
   1132             self.format(format)
   1133         if path is None:
-> 1134             self._jwrite.save()
   1135         else:
   1136             self._jwrite.save(path)

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python

In [0]:
#testing to ingestion
df_airlines.write.format("snowflake").options(**sfOptions).option("dbtable","airlines").mode("Overwrite").save()

In [0]:

#Total number of flights by airline and airport on a monthly basis
df_tot_flight.write.format("snowflake").options(**sfOptions).option("dbtable","tot_no_flgt_aline_aport").mode("Overwrite").save()
#On-time percentage of each airline for the year 2015
df_on_time.write.format("snowflake").options(**sfOptions).option("dbtable","on_time_perc_aline").mode("Overwrite").save()
df_largest_delay_airline.write.format("snowflake").options(**sfOptions).option("dbtable","larg_delay_aline").mode("Overwrite").save()
#Cancellation reasons by airport
df_derive_canc_reason.write.format("snowflake").options(**sfOptions).option("dbtable","canc_reason_aport").mode("Overwrite").save()
#Delay reasons by airport 
df_final_delay_rsn.write.format("snowflake").options(**sfOptions).option("dbtable","delay_reason_aport").mode("Overwrite").save()

In [0]:
df_tot_no_flgt_aline_aport = spark.read \
  .format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "tot_no_flgt_aline_aport") \
  .load()
display(df_tot_no_flgt_aline_aport)

YEAR,MONTH,AIRLINE,AIRPORT,TOTAL
2015,1,Alaska Airlines Inc.,Adak Airport,18
2015,1,Alaska Airlines Inc.,Albuquerque International Sunport,62
2015,1,Alaska Airlines Inc.,Austin-Bergstrom International Airport,62
2015,1,Alaska Airlines Inc.,Baltimore-Washington International Airport,62
2015,1,Alaska Airlines Inc.,Bellingham International Airport,189
2015,1,Alaska Airlines Inc.,Bethel Airport,164
2015,1,Alaska Airlines Inc.,Bob Hope Airport (Hollywood Burbank Airport),186
2015,1,Alaska Airlines Inc.,Chicago O'Hare International Airport,198
2015,1,Alaska Airlines Inc.,Dallas/Fort Worth International Airport,211
2015,1,Alaska Airlines Inc.,Deadhorse Airport (Prudhoe Bay Airport),160


In [0]:
df_on_time_perc_aline = spark.read \
  .format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "on_time_perc_aline") \
  .load()
display(df_on_time_perc_aline)

AIRLINE,ON_TIME_PERC
Alaska Airlines Inc.,0.563
American Airlines Inc.,1.768
American Eagle Airlines Inc.,5.635
Atlantic Southeast Airlines,3.016
Delta Air Lines Inc.,0.761
Frontier Airlines Inc.,0.796
Hawaiian Airlines Inc.,0.598
JetBlue Airways,2.14
Skywest Airlines Inc.,1.879
Southwest Airlines Co.,1.644


In [0]:
df_larg_delay_aline = spark.read \
  .format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "larg_delay_aline") \
  .load()
display(df_larg_delay_aline)

AIRLINE,CNT
Southwest Airlines Co.,38898


In [0]:
df_canc_reason_aport = spark.read \
  .format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "canc_reason_aport") \
  .load()
display(df_canc_reason_aport)

AIRPORT,CANCELLATION_REASON
Aberdeen Regional Airport,Airline/Carrier
Aberdeen Regional Airport,Weather
Abilene Regional Airport,Weather
Abilene Regional Airport,Airline/Carrier
Abraham Lincoln Capital Airport,Airline/Carrier
Abraham Lincoln Capital Airport,Weather
Abraham Lincoln Capital Airport,National Air System
Adak Airport,Weather
Akron-Canton Regional Airport,Weather
Akron-Canton Regional Airport,National Air System


In [0]:
df_delay_reason_aport = spark.read \
  .format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "delay_reason_aport") \
  .load()

display(df_delay_reason_aport)

java.lang.ClassCastException: java.lang.Long cannot be cast to java.lang.String
	at com.databricks.backend.daemon.driver.JsonUtils$JsonUtilTypeConverters$.$anonfun$makeConverter$7(JsonUtils.scala:202)
	at com.databricks.backend.daemon.driver.JsonUtils$JsonUtilTypeConverters$.$anonfun$makeStructConverter$3(JsonUtils.scala:256)
	at com.databricks.backend.daemon.driver.JsonUtils$JsonUtilTypeConverters$.$anonfun$makeConverter$7(JsonUtils.scala:202)
	at com.databricks.backend.daemon.driver.JsonUtils$JsonUtilTypeConverters$.$anonfun$makeRowToListConverter$4(JsonUtils.scala:114)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:459)
	at scala.collection.Iterator$$anon$15.hasNext(Iterator.scala:654)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$plus$plus$eq$(Growable.scala:53)
	at scala.collection.mutable.ListBuffer.$plus$plus$eq(ListBuffer.scala:184)
	at scala.collection.mutable.ListBuffer.$plus$plus$eq(ListBuffer.scala:47)
	at scala.collection.TraversableOnce.to(TraversableOnce.scala:315)
	at scala.collection.TraversableOnce.to$(TraversableOnce.scala:313)
	at scala.collection.AbstractIterator.to(Iterator.scala:1429)
	at scala.collection.TraversableOnce.toList(TraversableOnce.scala:299)
	at scala.collection.TraversableOnce.toList$(TraversableOnce.scala:299)
	at scala.collection.AbstractIterator.toList(Iterator.scala:1429)
	at com.databricks.backend.daemon.driver.JsonUtils$.trimResults(JsonUtils.scala:81)
	at com.databricks.backend.daemon.driver.OutputAggregator$.maybeApplyOutputAggregation(OutputAggregator.scala:431)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation0(OutputAggregator.scala:276)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation(OutputAggregator.scala:98)
	at com.databricks.backend.daemon.driver.PythonDriverLocalBase.generateTableResult(PythonDriverLocalBase.scala:524)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.computeListResultsItem(JupyterDriverLocal.scala:761)
	at com.databricks.backend.daemon.driver.PythonDriverLocalBase.genListResults(PythonDriverLocalBase.scala:488)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.repl(JupyterDriverLocal.scala:497)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$13(DriverLocal.scala:541)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:266)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:261)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:258)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:50)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:305)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:297)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:50)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:518)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:689)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:681)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:522)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:634)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:427)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:370)
	at com.databricks.backend.daemon.driv

In [0]:
df_most_uniq_routes_aline = spark.read \
  .format("snowflake") \
  .options(**sfOptions) \
  .option("dbtable", "most_uniq_routes_aline") \
  .load()
display(df_most_uniq_routes_aline)

AIRLINE,CNT
American Airlines Inc.,1
American Eagle Airlines Inc.,1
Atlantic Southeast Airlines,1
Delta Air Lines Inc.,1
Frontier Airlines Inc.,1
Skywest Airlines Inc.,1
US Airways Inc.,1
United Air Lines Inc.,1
Virgin America,1
American Airlines Inc.,2
